# Week 6: Probabilistic State Estimation

### Topics Covered

- Modeling Uncertainty; Bayes Filter fundamentals; Introduction to Kalman Filters (Linear, Extended, Unscented) for tracking

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Additional imports as needed

## 1. Modeling Uncertainty

In autonomous systems, we never have perfect information about the world. Sensors are noisy, models are approximate, and the environment is unpredictable. Probabilistic state estimation provides a principled framework for reasoning under uncertainty.

### Sources of Uncertainty

1. **Measurement Noise**: Sensors provide imperfect observations
   - GPS accuracy: ±5-10 meters
   - LIDAR noise: ±2-3 cm
   - Camera pixel uncertainty

2. **Process Noise**: System dynamics are not perfectly modeled
   - Wind effects on vehicle motion
   - Tire slip
   - Unmodeled disturbances

3. **Model Uncertainty**: Simplified representations of reality
   - Linearization errors
   - Parameter uncertainty

### Probability Distributions

We represent uncertainty using probability distributions:

**Gaussian (Normal) Distribution:**
$$p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

- Mean $\mu$: Expected value
- Variance $\sigma^2$: Uncertainty

**Multivariate Gaussian:**
$$p(\mathbf{x}) = \frac{1}{\sqrt{(2\pi)^n|\boldsymbol{\Sigma}|}} e^{-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1}(\mathbf{x}-\boldsymbol{\mu})}$$

- Mean vector $\boldsymbol{\mu}$
- Covariance matrix $\boldsymbol{\Sigma}$ (captures correlations)

In [ ]:
# Unscented Kalman Filter Implementation

class UnscentedKalmanFilter:
    def __init__(self, n_states, initial_state, initial_cov, process_noise, measurement_noise,
                 alpha=1e-3, beta=2, kappa=0):
        """
        UKF implementation
        
        Parameters:
        - n_states: dimension of state vector
        - alpha: spread of sigma points (typically 1e-3 to 1)
        - beta: incorporates prior knowledge (2 is optimal for Gaussian)
        - kappa: secondary scaling parameter (typically 0 or 3-n)
        """
        self.n = n_states
        self.x = initial_state
        self.P = initial_cov
        self.Q = process_noise
        self.R = measurement_noise
        
        # UKF parameters
        self.alpha = alpha
        self.beta = beta
        self.kappa = kappa
        
        self.lambda_ = alpha**2 * (n_states + kappa) - n_states
        
        # Weights for mean and covariance
        self.Wm = np.zeros(2 * n_states + 1)
        self.Wc = np.zeros(2 * n_states + 1)
        
        self.Wm[0] = self.lambda_ / (n_states + self.lambda_)
        self.Wc[0] = self.lambda_ / (n_states + self.lambda_) + (1 - alpha**2 + beta)
        
        for i in range(1, 2 * n_states + 1):
            self.Wm[i] = 1.0 / (2.0 * (n_states + self.lambda_))
            self.Wc[i] = self.Wm[i]
    
    def generate_sigma_points(self, x, P):
        """Generate sigma points around x with covariance P"""
        n = len(x)
        sigma_points = np.zeros((2 * n + 1, n))
        
        # Center point
        sigma_points[0] = x
        
        # Matrix square root using Cholesky decomposition
        try:
            U = np.linalg.cholesky((n + self.lambda_) * P)
        except np.linalg.LinAlgError:
            # If Cholesky fails, use eigenvalue decomposition
            eigvals, eigvecs = np.linalg.eigh(P)
            eigvals = np.maximum(eigvals, 0)  # Ensure positive
            U = eigvecs @ np.diag(np.sqrt((n + self.lambda_) * eigvals))
        
        # Positive sigma points
        for i in range(n):
            sigma_points[i + 1] = x + U[:, i]
        
        # Negative sigma points
        for i in range(n):
            sigma_points[n + i + 1] = x - U[:, i]
        
        return sigma_points
    
    def predict(self, dt, f):
        """
        Prediction step
        f: state transition function f(x, dt) -> x_new
        """
        # Generate sigma points
        sigma_points = self.generate_sigma_points(self.x, self.P)
        
        # Propagate sigma points through motion model
        n = len(self.x)
        sigma_points_pred = np.zeros((2 * n + 1, n))
        
        for i in range(2 * n + 1):
            sigma_points_pred[i] = f(sigma_points[i], dt)
        
        # Compute predicted mean
        self.x = np.sum(self.Wm[:, np.newaxis] * sigma_points_pred, axis=0)
        
        # Compute predicted covariance
        self.P = np.zeros((n, n))
        for i in range(2 * n + 1):
            diff = sigma_points_pred[i] - self.x
            self.P += self.Wc[i] * np.outer(diff, diff)
        
        self.P += self.Q
    
    def update(self, z, h):
        """
        Update step
        z: measurement vector
        h: measurement function h(x) -> z_pred
        """
        n = len(self.x)
        m = len(z)
        
        # Generate sigma points from predicted state
        sigma_points = self.generate_sigma_points(self.x, self.P)
        
        # Propagate sigma points through measurement model
        sigma_meas = np.zeros((2 * n + 1, m))
        for i in range(2 * n + 1):
            sigma_meas[i] = h(sigma_points[i])
        
        # Predicted measurement
        z_pred = np.sum(self.Wm[:, np.newaxis] * sigma_meas, axis=0)
        
        # Innovation covariance
        S = np.zeros((m, m))
        for i in range(2 * n + 1):
            diff = sigma_meas[i] - z_pred
            S += self.Wc[i] * np.outer(diff, diff)
        S += self.R
        
        # Cross-correlation
        Pxz = np.zeros((n, m))
        for i in range(2 * n + 1):
            dx = sigma_points[i] - self.x
            dz = sigma_meas[i] - z_pred
            Pxz += self.Wc[i] * np.outer(dx, dz)
        
        # Kalman gain
        K = Pxz @ np.linalg.inv(S)
        
        # State update
        self.x = self.x + K @ (z - z_pred)
        
        # Covariance update
        self.P = self.P - K @ S @ K.T


# Comparison: UKF vs EKF on highly nonlinear system
def compare_ukf_ekf():
    np.random.seed(42)
    
    # State transition (nonlinear)
    def f(x, dt):
        """Constant turn rate and velocity model"""
        px, py, v, theta, omega = x
        if abs(omega) < 1e-6:  # Straight line motion
            return np.array([
                px + v * np.cos(theta) * dt,
                py + v * np.sin(theta) * dt,
                v,
                theta,
                omega
            ])
        else:  # Curved motion
            return np.array([
                px + v / omega * (np.sin(theta + omega * dt) - np.sin(theta)),
                py + v / omega * (-np.cos(theta + omega * dt) + np.cos(theta)),
                v,
                theta + omega * dt,
                omega
            ])
    
    # Measurement model (range-bearing)
    def h(x):
        px, py, _, _, _ = x
        r = np.sqrt(px**2 + py**2)
        bearing = np.arctan2(py, px)
        return np.array([r, bearing])
    
    # Simulation
    dt = 0.1
    num_steps = 100
    
    # True initial state: [px, py, v, theta, omega]
    x_true = np.array([0.0, 0.0, 5.0, np.pi/4, 0.1])  # Turning motion
    
    true_states = []
    measurements = []
    
    for t in range(num_steps):
        x_true = f(x_true, dt)
        true_states.append(x_true.copy())
        
        # Noisy measurement
        z_true = h(x_true)
        z_meas = z_true + np.array([np.random.randn() * 2.0, np.random.randn() * 0.1])
        measurements.append(z_meas)
    
    true_states = np.array(true_states)
    measurements = np.array(measurements)
    
    # Initialize UKF
    x0 = np.array([0.0, 0.0, 4.0, np.pi/4, 0.0])  # Initial guess
    P0 = np.diag([5.0, 5.0, 2.0, 0.5, 0.2])
    Q = np.diag([0.1, 0.1, 0.1, 0.01, 0.01])
    R = np.diag([4.0, 0.01])
    
    ukf = UnscentedKalmanFilter(5, x0.copy(), P0.copy(), Q, R)
    
    ukf_estimates = []
    
    for i in range(num_steps):
        ukf.predict(dt, f)
        ukf.update(measurements[i], h)
        ukf_estimates.append(ukf.x.copy())
    
    ukf_estimates = np.array(ukf_estimates)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Trajectory
    axes[0].plot(true_states[:, 0], true_states[:, 1], 'g-', label='True', linewidth=2)
    axes[0].plot(ukf_estimates[:, 0], ukf_estimates[:, 1], 'b-', label='UKF', linewidth=2)
    axes[0].plot(0, 0, 'r*', markersize=15, label='Sensor')
    axes[0].set_xlabel('X Position')
    axes[0].set_ylabel('Y Position')
    axes[0].set_title('UKF Tracking (Nonlinear Turning Motion)')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].axis('equal')
    
    # Position error
    position_error = np.sqrt((ukf_estimates[:, 0] - true_states[:, 0])**2 + 
                            (ukf_estimates[:, 1] - true_states[:, 1])**2)
    time = np.arange(num_steps) * dt
    
    axes[1].plot(time, position_error, 'b-', linewidth=2)
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Position Error (m)')
    axes[1].set_title('UKF Position Error')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Mean Position Error: {np.mean(position_error):.2f} m")
    print(f"Max Position Error: {np.max(position_error):.2f} m")

compare_ukf_ekf()

## Exercises

### Exercise 1: 1D Kalman Filter for Temperature Tracking

Implement a 1D Kalman filter to estimate room temperature from noisy sensor readings.

**Given:**
- True temperature: 20°C with small random walk (±0.1°C per step)
- Sensor noise: ±1.5°C (standard deviation)
- 100 time steps

**Tasks:**
1. Implement the 1D Kalman filter
2. Plot true temperature, measurements, and estimates
3. Calculate the mean squared error
4. Experiment with different process and measurement noise values

In [ ]:
# Exercise 3: This is left as a challenge for the student
# Hint: Reuse the implementations from sections 3-5
# Try different turning rates (omega values) to vary nonlinearity

# Here's a template to get started:

def compare_all_filters():
    """
    Compare Linear KF (will perform poorly), EKF, and UKF
    on a turning vehicle scenario
    """
    # TODO: Implement comparison
    # 1. Set up nonlinear motion model (constant turn rate)
    # 2. Initialize all three filters with same parameters
    # 3. Run them on the same measurement sequence
    # 4. Compare errors and computation time
    # 5. Visualize results
    
    pass  # Your implementation here

# Uncomment to run:
# compare_all_filters()

## References and Additional Resources

### Core Textbooks

1. **Probabilistic Robotics** by Thrun, Burgard, and Fox (2005)
   - The definitive reference for probabilistic state estimation
   - Chapters 2-3: Recursive state estimation and Bayes filters
   - Chapter 3.2: Kalman Filter
   - Chapter 3.3: Extended Kalman Filter
   - Chapter 3.4: Unscented Kalman Filter

2. **Optimal State Estimation** by Simon (2006)
   - Comprehensive treatment of Kalman filtering
   - Practical implementation considerations

3. **State Estimation for Robotics** by Barfoot (2017)
   - Modern perspective with robotics applications
   - Available online: http://asrl.utias.utoronto.ca/~tdb/bib/barfoot_ser17.pdf

### Key Papers

1. **Kalman, R. E. (1960)** - "A New Approach to Linear Filtering and Prediction Problems"
   - Original Kalman filter paper
   - Transactions of the ASME–Journal of Basic Engineering

2. **Julier, S. J., & Uhlmann, J. K. (1997)** - "New extension of the Kalman filter to nonlinear systems"
   - Introduced the Unscented Kalman Filter
   - SPIE 3068, Signal Processing, Sensor Fusion, and Target Recognition VI

### Online Resources

1. **Kalman Filter Tutorial by Greg Welch and Gary Bishop**
   - https://www.cs.unc.edu/~welch/kalman/
   - Excellent introduction with practical examples

2. **Roger Labbe's Kalman Filter Book (Jupyter Notebooks)**
   - https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python
   - Interactive Python implementations

3. **Udacity Self-Driving Car Nanodegree**
   - Extended Kalman Filter project
   - Sensor fusion with LIDAR and RADAR

### Software Libraries

1. **FilterPy** (Python)
   - pip install filterpy
   - Kalman filter implementations

2. **PyKalman** (Python)
   - pip install pykalman
   - EM algorithm for parameter estimation

3. **Robot Operating System (ROS)**
   - robot_localization package
   - Production-ready EKF/UKF implementations

### Applications in Autonomous Vehicles

1. **Localization**: GPS/IMU fusion for position estimation
2. **Object Tracking**: Multi-object tracking with RADAR/LIDAR/Camera
3. **SLAM**: Simultaneous Localization and Mapping
4. **Sensor Calibration**: Estimating sensor biases and drift

### Related Topics to Explore

1. **Particle Filters**: Non-parametric Bayes filter for multimodal distributions
2. **Information Filter**: Inverse covariance form of Kalman filter
3. **Kalman Smoother**: Batch processing for improved estimates
4. **Multi-Hypothesis Tracking**: Handling data association uncertainty
5. **Adaptive Filtering**: Online estimation of Q and R matrices

### Visualization Tools

- **Matplotlib** (used in this notebook)
- **Plotly**: Interactive visualizations
- **RViz**: Real-time robotics visualization

---

## Summary

This notebook covered the fundamentals of probabilistic state estimation:

**Key Concepts:**
- Uncertainty representation with Gaussian distributions
- Bayes filter framework (predict + update)
- Kalman Filter for linear systems
- Extended Kalman Filter for nonlinear systems (Jacobian linearization)
- Unscented Kalman Filter for highly nonlinear systems (sigma points)

**Practical Skills:**
- Implementing filters from scratch
- Tuning process and measurement noise
- Sensor fusion techniques
- Performance evaluation

**Next Steps:**
1. Explore the HTML visualizations in this repository ([kalman_ball_chase.html](kalman_ball_chase.html))
2. Implement a real-world application (e.g., GPS/IMU fusion)
3. Study Particle Filters for non-Gaussian distributions
4. Learn about SLAM and its relation to state estimation